# Pandas - Merge & Group by
Answer the questions below.


*   a) What is the full name of the state with the highest "total" population in 2012?
*   b) If we only consider the "total" population, what is the state with the highest population per area (sq. mi) in 2012?
* c) which area did get a pick-up(pickup_borough) the most?
* d) what is the average fare in each pickup_borough area?

In [1]:
# Following are shell commands to download the data
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0  81544      0 --:--:-- --:--:-- --:--:-- 81598
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0   1760      0 --:--:-- --:--:-- --:--:--  1757
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   1896      0 --:--:-- --:--:-- --:--:--  1899


In [2]:
import pandas as pd
import seaborn as sns

###### a,b #######
pop = pd.read_csv('/content/state-population.csv')
areas = pd.read_csv('/content/state-areas.csv')
abbrevs = pd.read_csv('/content/state-abbrevs.csv')
pd.set_option('display.max_columns', 74)
pd.set_option('display.width', 500)

print(pop.head())
print(areas.head())
print(abbrevs.head())


###### c,d #######
taxisDf = sns.load_dataset('taxis')

print(taxisDf.info())
print(taxisDf.head())


  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6433 entries, 0 to 6432
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   pickup           6433 non-null   datetime64[ns]
 1   dropoff          6433 non-null   datetime64[ns]
 2   passengers       6433 non-null   int64         
 3   distance         6433 non-null   fl

In [26]:
pop_total = pop[(pop['year'] == 2012) & (pop['ages'] == 'total') & (pop['state/region'] !='USA') & (pop['state/region'] !='PR')] # Select only state with total pop and left the USA
pop_max_2012 = pop_total[(pop_total['population'] == pop_total['population'].max())] #Select the max pop number by .max()
merge_df = pd.merge(pop_max_2012,abbrevs,how = 'left', left_on = 'state/region', right_on= 'abbreviation') #merge df and to match the abreviation and state
state_name_max_pop = merge_df['state'].iloc[0] #Select onlt state name
print(state_name_max_pop)
print(merge_df)

California
  state/region   ages  year  population       state abbreviation
0           CA  total  2012  37999878.0  California           CA


In [32]:
pop_total_area = pd.merge(pop_total,abbrevs,how = 'left',left_on = 'state/region',right_on = 'abbreviation')

pop_total_area = pd.merge(pop_total_area,areas,how = 'left',left_on = 'state',right_on = 'state')

pop_total_area['Pop per Area'] = pop_total_area['population']/pop_total_area['area (sq. mi)']
print(pop_total_area)


   state/region   ages  year  population                 state abbreviation  area (sq. mi)  Pop per Area
0            AL  total  2012   4817528.0               Alabama           AL          52423     91.897221
1            AK  total  2012    730307.0                Alaska           AK         656425      1.112552
2            AZ  total  2012   6551149.0               Arizona           AZ         114006     57.463195
3            AR  total  2012   2949828.0              Arkansas           AR          53182     55.466662
4            CA  total  2012  37999878.0            California           CA         163707    232.121278
5            CO  total  2012   5189458.0              Colorado           CO         104100     49.850701
6            CT  total  2012   3591765.0           Connecticut           CT           5544    647.865260
7            DE  total  2012    917053.0              Delaware           DE           1954    469.320880
8            DC  total  2012    633427.0  District of C

In [36]:
max_per_area = pop_total_area[(pop_total_area['Pop per Area'] == pop_total_area['Pop per Area'].max())]
print(max_per_area['state'].iloc[0])

District of Columbia


In [65]:
pickup_counts_pivot = taxisDf.pivot_table(index='pickup_borough', aggfunc='size')
Most_Pickup = pickup_counts_pivot.idxmax()
print(Most_Pickup)

Manhattan


In [66]:
avg_pivot = taxisDf.pivot_table(index = 'pickup_borough',values = 'fare',aggfunc = 'mean')
print(avg_pivot)

                     fare
pickup_borough           
Bronx           20.999091
Brooklyn        16.520836
Manhattan       11.152889
Queens          24.934642
